<a href="https://colab.research.google.com/github/Kolawole-a2/Kola_Projects/blob/main/AFOLABI_SEAS_8410_DC8_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📌 **Notebook Summary: CISO Dashboard Wireframe**

This notebook provides an interactive dashboard for visualizing vulnerability datasets using Dash within Google Colab. It allows a user to upload a CSV file, explore the data through interactive buttons, and automatically generate downloadable PNG visualizations.


✅ **Key Features:**
CSV Upload: Upload any vulnerability dataset in CSV format.

Data Cleaning: Automatically cleans and standardizes column names for consistency.


**Interactive Dashboard:**

Displays a dashboard titled "CISO Dashboard Wireframe" with:

A. Three filter placeholder buttons (for UI structure).

B. Five actionable buttons for key insights:

1. Vulnerability Count by Severity

2. Overall Vulnerability Count

3. Open/Close Trends

4. FISMA Counts

5. Category Counts

**PNG Graph Saving:** Every graph generated through a button click can be automatically saved as a PNG file to the user's local system by clicking on the camera image at the top of each graph.

**Download Option:** You can manually download saved PNG files after each interaction.

**Inline Display:** The dashboard runs inline within the notebook for ease of use.

💡 **How to Use:**
Run All Cells Sequentially to:

- Install required packages

- Upload your CSV

- Launch the dashboard

**Click Dashboard Buttons to:**

See interactive charts or count summaries

Save each chart as a PNG file using the camera image among the list of icons displayed at the top of each graph


🚨 **Important Notes:**
The dashboard requires specific columns like severity, status, fisma, and category in your CSV.

If a column is missing, it shows a user-friendly error message.

The dashboard uses a clean brown color theme for buttons and a centered layout for clarity.




In [2]:
!pip install jupyter-dash dash pandas plotly kaleido --quiet

import pandas as pd
from dash import Dash, html, Output, Input, State, dcc, ctx
from jupyter_dash import JupyterDash
import plotly.express as px
import io

from google.colab import files
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[file_name]))
df.dropna(how='all', inplace=True)
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

print("📊 Cleaned Data Sample:")
print(df.head())

app = JupyterDash(__name__)

button_style = {
    'background-color': '#8B4513',
    'color': 'white',
    'padding': '20px',
    'margin': '10px',
    'border': 'none',
    'font-size': '20px',
    'width': '250px',
    'border-radius': '10px',
    'text-align': 'center',
    'cursor': 'pointer'
}

app.layout = html.Div([
    html.H1("CISO Dashboard Wireframe", style={'textAlign': 'center', 'color': '#4B2E83'}),

    html.Div([
        html.Button('Filter Option 1', id='filter-1', n_clicks=0, style=button_style),
        html.Button('Filter Option 2', id='filter-2', n_clicks=0, style=button_style),
        html.Button('Filter Option 3', id='filter-3', n_clicks=0, style=button_style),
    ], style={'display': 'flex', 'justify-content': 'center', 'margin-bottom': '20px'}),

    html.Div([
        html.Button('Vulnerability Count by Severity', id='severity-btn', n_clicks=0, style=button_style),
        html.Button('Overall Vulnerability Count', id='overall-btn', n_clicks=0, style=button_style),
        html.Button('Open/Close Trends', id='trend-btn', n_clicks=0, style=button_style),
        html.Button('FISMA Counts', id='fisma-btn', n_clicks=0, style=button_style),
        html.Button('Category Counts', id='category-btn', n_clicks=0, style=button_style),
    ], style={'display': 'flex', 'flex-wrap': 'wrap', 'justify-content': 'center'}),

    html.Div(id='output-div', style={'textAlign': 'center', 'margin-top': '30px'})
])

@app.callback(
    Output('output-div', 'children'),
    [Input('severity-btn', 'n_clicks'),
     Input('overall-btn', 'n_clicks'),
     Input('trend-btn', 'n_clicks'),
     Input('fisma-btn', 'n_clicks'),
     Input('category-btn', 'n_clicks')],
    prevent_initial_call=True
)
def update_output(sev, overall, trend, fisma, category):
    triggered = ctx.triggered_id

    if triggered == 'severity-btn':
        if 'severity' in df.columns:
            counts = df['severity'].value_counts().to_frame().reset_index()
            counts.columns = ['Severity', 'Count']
            fig = px.bar(counts, x='Severity', y='Count', title='Vulnerability Count by Severity')
            fig.write_image("severity_chart.png")
            print("✅ PNG saved as severity_chart.png")
            return dcc.Graph(figure=fig)
        else:
            return "❌ 'severity' column not found in data."

    elif triggered == 'overall-btn':
        count = len(df)
        return html.H2(f"✅ Overall Vulnerability Count: {count}")

    elif triggered == 'trend-btn':
        if 'status' in df.columns:
            trend = df['status'].value_counts().to_frame().reset_index()
            trend.columns = ['Status', 'Count']
            fig = px.bar(trend, x='Status', y='Count', title='Open/Close Trends')
            fig.write_image("trend_chart.png")
            print("✅ PNG saved as trend_chart.png")
            return dcc.Graph(figure=fig)
        else:
            return "❌ 'status' column not found in data."

    elif triggered == 'fisma-btn':
        fisma_cols = [col for col in df.columns if 'fisma' in col]
        if fisma_cols:
            fisma_counts = df[fisma_cols[0]].value_counts().to_frame().reset_index()
            fisma_counts.columns = ['FISMA', 'Count']
            fig = px.bar(fisma_counts, x='FISMA', y='Count', title='FISMA Counts')
            fig.write_image("fisma_chart.png")
            print("✅ PNG saved as fisma_chart.png")
            return dcc.Graph(figure=fig)
        else:
            return "❌ No FISMA-related column found."

    elif triggered == 'category-btn':
        if 'category' in df.columns:
            cat_counts = df['category'].value_counts().to_frame().reset_index()
            cat_counts.columns = ['Category', 'Count']
            fig = px.bar(cat_counts, x='Category', y='Count', title='Category Counts')
            fig.write_image("category_chart.png")
            print("✅ PNG saved as category_chart.png")
            return dcc.Graph(figure=fig)
        else:
            return "❌ 'category' column not found in data."

    return "❓ Click a command button to see data insights."

app.run(mode='inline')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 592.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 1.7 MB/s eta 0:00:00


Saving HW5FortifyOC.csv to HW5FortifyOC (1).csv
📊 Cleaned Data Sample:
                           vulnerability_details_url application  \
0  https://trial.fortify.com//Redirect/Issues/909...   AndroGoat   
1  https://trial.fortify.com//Redirect/Issues/add...   AndroGoat   
2  https://trial.fortify.com//Redirect/Issues/b32...   AndroGoat   
3  https://trial.fortify.com//Redirect/Issues/983...   AndroGoat   
4  https://trial.fortify.com//Redirect/Issues/d4f...   AndroGoat   

   application_id    release  release_id      fisma     stig39   stig_4.1  \
0          202682  4/21/2019      205852  (Not Set)  (Not Set)  (Not Set)   
1          202682  4/21/2019      205852  (Not Set)  (Not Set)  (Not Set)   
2          202682  4/21/2019      205852  (Not Set)  (Not Set)  (Not Set)   
3          202682  4/21/2019      205852  (Not Set)  (Not Set)  (Not Set)   
4          202682  4/21/2019      205852  (Not Set)  (Not Set)  (Not Set)   

    stig_4.3  stig_4.11  ...  source scan_tool    sink da

/usr/local/lib/python3.11/dist-packages/dash/dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

This notebook simplifies data analysis for cybersecurity or vulnerability management teams by combining interactive dashboards, automatic chart exports, and fast reporting — all in one place.